# R, Data Cleaning
As you continue working with data, though, you'll find that much of the data you encounter will be messy, and lots of your time will be spent preparing it for analysis. These preparations, known as data cleaning, include:

1. Removing data you don't need for your analysis
2. Removing duplicate data
3. Dealing with missing data and outliers
4. Creating new variables where necessary
5. Combining separate data sets

Data cleaning is often a non-linear process. You'll need to spend time researching your data, use your professional judgement to make decisions, and sometimes revisit your data cleaning choices mid-project. That's okay: Data cleaning is a crucial part of being a data analyst or scientist, and getting good at it takes practice.

## New York City Schools Data

The data sets you'll be working with in this course come from the New York City Department of Education. They contain data on NYC schools, including student demographics, test scores, graduation and dropout rates, and school locations.

These data sets contain interesting information that we can use to learn about the characteristics of schools located throughout a large, diverse city.

We'll focus on using the data to understand how NYC high schools' demographics (race, sex, income, etc.) affect how well students perform academically.

You'll have a chance to analyze the data to answer questions such as:

1. Is there a relationship between school class size and test performance?
2. Are there any demographic factors that seem to be related to student test performance?
3. Does academic performance vary by school location?

The data sets are stored in six files:

1. sat_results.csv 
    - Data on Scholastic Aptitude Test (SAT) scores. The SAT is a standardized test taken by most high school students that is used to assess preparation for college.


2. ap_2010.csv
    - Data on Advanced Placement (AP) test results. AP tests are taken by advanced high school students, often at schools that can afford to provide the specialized classes needed to prepare for them.


3. class_size.csv
    - Data on average school class size.


4. demographics.csv
    - Data on demographics of NYC school students, including race and sex.


5. graduation.csv
    - Data on graduation outcomes, including percentages of students who graduated or dropped out of school.
    

6. hs_directory.csv
    - A directory of high schools that contains location information for each school.

In [33]:
#import packages
library(readr)
library(dplyr)
library(ggplot2)
library(purrr)

In [34]:
# read SAT file
sat_results <- read_csv("sat_results.csv")

Parsed with column specification:
cols(
  DBN = col_character(),
  `SCHOOL NAME` = col_character(),
  `Num of SAT Test Takers` = col_character(),
  `SAT Critical Reading Avg. Score` = col_character(),
  `SAT Math Avg. Score` = col_character(),
  `SAT Writing Avg. Score` = col_character()
)


In [35]:
# read AP Stat file
ap_2010 <- read_csv("ap_2010.csv")

Parsed with column specification:
cols(
  DBN = col_character(),
  SchoolName = col_character(),
  `AP Test Takers` = col_character(),
  `Total Exams Taken` = col_character(),
  `Number of Exams with scores 3 4 or 5` = col_character()
)


In [36]:
# read Class Size file
class_size <- read_csv("class_size.csv")

Parsed with column specification:
cols(
  CSD = col_double(),
  BOROUGH = col_character(),
  `SCHOOL CODE` = col_character(),
  `SCHOOL NAME` = col_character(),
  GRADE = col_character(),
  `PROGRAM TYPE` = col_character(),
  `CORE SUBJECT (MS CORE and 9-12 ONLY)` = col_character(),
  `CORE COURSE (MS CORE and 9-12 ONLY)` = col_character(),
  `SERVICE CATEGORY(K-9* ONLY)` = col_character(),
  `NUMBER OF STUDENTS / SEATS FILLED` = col_double(),
  `NUMBER OF SECTIONS` = col_double(),
  `AVERAGE CLASS SIZE` = col_double(),
  `SIZE OF SMALLEST CLASS` = col_double(),
  `SIZE OF LARGEST CLASS` = col_double(),
  `DATA SOURCE` = col_character(),
  `SCHOOLWIDE PUPIL-TEACHER RATIO` = col_double()
)


In [37]:
# read demogrpahics file
demographics <- read_csv("demographics.csv")

Parsed with column specification:
cols(
  .default = col_double(),
  DBN = col_character(),
  Name = col_character()
)
See spec(...) for full column specifications.


In [38]:
# read graduation file
graduation <- read_csv("graduation.csv")

Parsed with column specification:
cols(
  .default = col_character(),
  `Total Cohort` = col_double()
)
See spec(...) for full column specifications.


In [39]:
# read graduation file
hs_directory <- read_csv("hs_directory.csv")

Parsed with column specification:
cols(
  .default = col_character(),
  grade_span_max = col_double(),
  expgrade_span_max = col_double(),
  zip = col_double(),
  start_time = col_time(format = ""),
  end_time = col_time(format = ""),
  number_programs = col_double()
)
See spec(...) for full column specifications.


#### Which data do we need for our analysis?
For example, we are interested in data for NYC high schools. However, the class_size and demographics data frames also contain information about elementary schools. Some of the data frames also contain observations from multiple years, and it may make sense to work with only the most recent years' data.

#### Do we need to create any new variables?
For example, the sat_results data frame contains variables for students' scores on sections of the SAT: Math, Reading, and Writing. However, for our analysis, we may want a total SAT score variable. We would need to create it by calculating the sum of the section scores.

#### Are the data of the correct type?
Are any of the data that you need to use to calculate new variables, for example, formatted as character instead of numeric?

#### Do we need to combine data frames?
To analyze the NYC high schools data, we ultimately want to be able to perform summary calculations and visualize relationships between variables to understand how demographic factors affect test performance. We will need to combine the six data frames into a single, clean data frame.

We can combine multiple data frames if they share a variable in common, and if that variable uniquely identifies observations. We call this variable a key.

In the case of these data frames, we see that nearly all of them share a variable in common: the DBN, or district borough number, that uniquely identifies each school.

#### Are there missing data?
In some data frames like demographics, there are many missing values (represented by NA). We will have to decide how to handle these missing values as we clean the data.

## SAT Data Set
We'll begin by assessing data cleaning needs for the sat_results data frame. Have a look at the sat_results data frame using the head() function to return the first ten rows.

Note that each variable name is surrounded in tick marks (\`\`). This is because the titles contain spaces, which are not valid in R variable names. When you work with tidyverse functions, surrounding invalid R variable names with tick marks will allow R to interpret them properly.

The data frame contains information about each of the 477 public schools in NYC. Since we know only high school students take the SAT, the fact that each school has SAT scores listed means that these are all high schools.

#### 1.1 Which data do we need for our analysis?
The data frame contains six variables:

1. Two (DBN and SCHOOL NAME) that provide information about the high school
2. One that tells us the number of students who took the SAT (Num of SAT Test Takers)
3. Three that tell us the average scores students at each high school earned on SAT sections (SAT Critical Reading Avg. Score, SAT Math Avg. Score, SAT Writing Avg. Score).

These variables are all relevant in the context of understanding the effect of demographics on test scores, so we will retain all the data in this data frame.

We can also see that there seems to be only one observation for each high school, so we will not need to remove any rows.

#### 1.2 Do we need to create any new variables? 
For our analysis, it will probably be useful to have a variable containing average total SAT scores. We can calculate this variable by taking the sum of the SAT section scores.

1. Critical Reading
2. Math
3. Writing

#### 1. 3 Are the data of the correct type?
To calculate the sum of the SAT section scores, we need the data to be numeric — we cannot perform arithmetic on character data. Since the data in the Num of SAT Test Takers, SAT Critical Reading Avg. Score, SAT Math Avg. Score, SAT Writing Avg. Score columns is character data, we need to change it to numeric.

#### 1.4 Do we have a key to combine multiple data frames?
Since this data frame contains a DBN variable, we can use it as a key to combine it with the other data frames.

In [40]:
# Change datatypes on columns from character to numeric using mutate

sat_results <- sat_results %>%
    mutate(`Num of SAT Test Takers` = as.numeric(`Num of SAT Test Takers`),
          `SAT Critical Reading Avg. Score` = as.numeric(`SAT Critical Reading Avg. Score`),
          `SAT Math Avg. Score` = as.numeric(`SAT Math Avg. Score`),
          `SAT Writing Avg. Score` = as.numeric(`SAT Writing Avg. Score`)) %>%
    mutate(avg_sat_score = `SAT Critical Reading Avg. Score`+  `SAT Math Avg. Score`+`SAT Writing Avg. Score`)

Warning message:
"NAs introduced by coercion"Warning message:
"NAs introduced by coercion"Warning message:
"NAs introduced by coercion"Warning message:
"NAs introduced by coercion"

## ap_2010 dataframe
For now, don't worry about warning messages like the following, which occur when some character values cannot be changed to numeric.

Now that we've identified and performed data cleaning operations for the sat_results data frame, let's have a look at the ap_2010 data frame and run through questions to help identify data cleaning needs.

#### Which data do we need for our analysis?
Since only high school students take AP exams, we know that all the data are for high schools. We do not need to remove any rows.

The data frame contains five variables. We will need DBN as a key to combine data frames. AP Test Takers provides information about the number of students who took one AP exam, and Total Exams Taken tells us how many exams were taken at the school (since students can take exams in multiple subjects). The variable Number of Exams with scores 3 4 or 5 tells us how many students earned high scores on the exams, which are scored on a scale from one (lowest) to five (highest). Since all these variables provide information about test scores, they are relevant for our analysis.

#### Do we need to create any new variables?
There are a few interesting variables we can create to help us understand these data better.

Since students can take multiple AP exams, it would be interesting to calculate the average number of exams taken per student by dividing Total Exams Taken by AP Test Takers.

We could also calculate the percentage of high-scoring AP exams by dividing Number of Exams with scores 3 4 or 5 by Total Exams Taken and multiplying by 100.

#### Are the data of the correct type?
As was the case for sat_results, the data in ap_2010 are all formatted as character data. We'll need to change them to numeric.

Before we perform the data cleaning operations on ap_2010, let's return for a moment to how you changed the data type of variables in sat_results.

To change the data types of the sat_results variables Num of SAT Test Takers, SAT Critical Reading Avg. Score, SAT Math Avg. Score, SAT Writing Avg. Score from character to numeric, you probably wrote code that looked like this:

    sat_results <- sat_results %>%
      mutate(`Num of SAT Test Takers` =  as.numeric(`Num of SAT Test Takers`),
             `SAT Writing Avg. Score` = as.numeric(`SAT Writing Avg. Score`), 
             `SAT Critical Reading Avg. Score` = as.numeric(`SAT Critical Reading Avg. Score`),
             `SAT Math Avg. Score` = as.numeric(`SAT Math Avg. Score`))
As you typed, advice from earlier courses may have come to mind: If you are copying and pasting repeatedly, there is probably a better coding solution.

In this case, a better solution exists in the form of the dplyr function mutate_at(). The mutate_at() function is a scoped variant of the mutate() function. This means you can use it to perform an operation on all variables within a specified subset.

Let's look at how we could change the data type of the four columns in sat_results using mutate_at() instead of mutate():

    sat_results <- sat_results %>%
      mutate_at(vars(`Num of SAT Test Takers`: `SAT Writing Avg. Score`), as.numeric)
Notice that instead of typing out the names of all the columns you want to modify, you can use a colon (:) to specify the range. This is useful in the event that you need to modify dozens of variables.

You can also specify the numbers of the columns you want to modify:

    sat_results <- sat_results %>%
      mutate_at(3:6, as.numeric)
When you use column numbers within mutate_at(), notice that you don't need to use vars() (which stands for "variables").

In [41]:
#Convert variables to numeric and add new columns
ap_2010 <- ap_2010 %>%
  mutate_at(3:5, as.numeric) %>%
  mutate(exams_per_student = `Total Exams Taken` / `AP Test Takers`) %>%
  mutate(high_score_percent = (`Number of Exams with scores 3 4 or 5`/`Total Exams Taken`)*100)

Warning message:
"NAs introduced by coercion"Warning message:
"NAs introduced by coercion"Warning message:
"NAs introduced by coercion"

## Class_size data frame

Information about class size, the variables AVERAGE CLASS SIZE, SIZE OF SMALLEST CLASS, SIZE OF LARGEST CLASS, may be useful in our analysis of how demographic factors may affect student academic performance. For example, wealthier schools may be able to offer smaller classes because they can afford more teachers.

One of the first things that's apparent when you look at class_size is that there are multiple rows for each school. Let's figure out why.

First, look at the GRADE column. It appears that each school has multiple rows for different grades. Since we are only interested in high schools, we can select only rows in this data set for which the GRADE variable has the value of "09-12" — grades that are considered part of high school in the U.S.

Next, look at the PROGRAM TYPE column. Some schools have several different types of programs. For consistency among data sets, let's only keep rows for which PROGRAM TYPE has the value of "GEN ED", which is short for "general education."

In [42]:
# Filter the class_size data frame to retain only rows where values of GRADE are equal to "09-12" and values of PROGRAM TYPE are "GEN ED".
class_size <- class_size %>%
  filter(GRADE == "09-12", `PROGRAM TYPE` == "GEN ED")

In [43]:
#Create a new class_size data frame that contains, CSD, SCHOOL CODE & SCHOOL NAME
class_size <- class_size %>%
    group_by(CSD, `SCHOOL CODE`, `SCHOOL NAME`) %>%
    summarize(avg_class_size = mean(`AVERAGE CLASS SIZE`),
              avg_largest_class = mean(`SIZE OF LARGEST CLASS`),
              avg_smallest_class = mean(`SIZE OF SMALLEST CLASS`))

The new class_size data frame is now simplified to contain the information for analysis. There is one row for each high school, and averages for smallest class size, largest class size, and average class size.

This data frame is nearly ready for analysis, but there's one more step we'll need to complete: Creating a DBN variable that we can use as a key to join the data frames.

In [44]:
library(stringr)

In [45]:
#Create a new DBN variable in the class_size data frame.
class_size <- class_size %>%
    mutate(DBN = str_c(CSD,`SCHOOL CODE`, sep = "")) %>%
    mutate(DBN = str_pad(DBN, width = 6, side = 'left', pad = '0'))

In [46]:
#check DBN column
head(class_size)

CSD,SCHOOL CODE,SCHOOL NAME,avg_class_size,avg_largest_class,avg_smallest_class,DBN
1,M292,Henry Street School for International Studies,22.56429,26.57143,18.50,01M292
1,M332,University Neighborhood Middle School,22.00000,23.50000,21.00,01M332
1,M378,School for Global Leaders,33.00000,33.00000,33.00,01M378
1,M448,University Neighborhood High School,22.23125,27.06250,18.25,01M448
1,M450,East Side Community School,21.20000,22.86667,19.40,01M450
1,M458,Forsyth Satellite Academy,23.00000,23.40000,22.60,01M458


## gradutation dataframe
Let's focus on graduation rates for each school's entire cohort by selecting rows for which the value of Demographic is "Total Cohort". We'll work with the most current graduation data, which is from 2006.

Do we need all the variables in the data frame for our analysis? To understand students' academic success, the most relevant variables are probably the ones that have to do with whether students successfully graduate or if they drop out: Total Grads - % of cohort and Dropped Out - % of cohort.


In [47]:
#Create a new graduation data frame that contains only Total Cohort and 2006 cohort
graduation <- graduation %>%
    filter(Demographic == 'Total Cohort' & Cohort == "2006") %>%
    select(DBN, `School Name`, `Total Grads - % of cohort`, `Dropped Out - % of cohort`)

## demographics dataframe
There are data for multiple years (the schoolyear variable). Let's work with data from the most recent year, rows for which values of schoolyear are "20112012".

The duplicate rows for each school also are due to the fact that data from elementary schools, as well as high schools, are included in demographics. There are columns (grade_1, grade_2, etc.) that contain the number of students in each grade for each school. Because we are only interested in working with high schools (grades 9, 10, 11, and 12), we can filter the data to select only schools for which the value of one of the high school grade columns.

I will only retain columns valuable for my analysis

In [48]:
#Create a new demographics data frame that contains only
demographics <- demographics %>%
    filter(schoolyear =="20112012" & grade9 != "NA") %>%
    select(DBN, Name, frl_percent, total_enrollment, ell_percent, sped_percent, asian_per, black_per, hispanic_per, white_per, male_per, female_per)

## hs_directory
The hs_directory data frame does contain a DBN variable to use as a key, but notice that it is named with lowercase letters (dbn). We will need to capitalize it.

In [49]:
hs_directory <- hs_directory %>%
    select(dbn, school_name, `Location 1`) %>%
    rename(DBN = dbn)

DBN,school_name,Location 1
17K548,Brooklyn School for Music & Theatre,"883 Classon Avenue Brooklyn, NY 11225 (40.67029890700047, -73.96164787599963)"
09X543,High School for Violin and Dance,"1110 Boston Road Bronx, NY 10456 (40.8276026690005, -73.90447525699966)"
09X327,Comprehensive Model School Project M.S. 327,"1501 Jerome Avenue Bronx, NY 10452 (40.842414068000494, -73.91616158599965)"
02M280,Manhattan Early College School for Advertising,"411 Pearl Street New York, NY 10038 (40.71067947100045, -74.00080702099967)"
28Q680,Queens Gateway to Health Sciences Secondary School,"160-20 Goethals Avenue Jamaica, NY 11432 (40.718810094000446, -73.80650045499965)"
08X348,Schuylerville Preparatory High School,"3000 East Tremont Avenue Bronx, NY 10461 (40.840513977000455, -73.83812095999963)"
14K474,PROGRESS High School for Professional Careers,"850 Grand Street Brooklyn, NY 11211 (40.71196311300048, -73.94043436699963)"
02M420,High School for Health Professions and Human Services,"345 East 15Th Street New York, NY 10003 (40.73248537800049, -73.98305338799963)"
28Q350,Jamaica Gateway to the Sciences,"167-01 Gothic Drive Jamaica, NY 11432 (40.713577459000476, -73.79651782499963)"
29Q313,Benjamin Franklin High School for Finance & Information Technology,"207-01 116th Avenue Cambria Heights, NY 11411 (40.6978073300005, -73.74585775499963)"


# Combining all dataframes
As we decided on necessary cleaning operations for each data frame, we ensured that each data frame had one observation - that is, one row - for each school. The DBN variable is unique to each school, and so to successfully combine the data frames, we should have no more than one instance of each value of DBN per data frame.

### removing duplicate vaules for DBN

In [50]:
#Create a list of the six data frames named ny_schools.
ny_schools <- list(sat_results, ap_2010, class_size, demographics, graduation, hs_directory)

#Assign the data frames in the list of name attributes.
names(ny_schools) <- c("sat_results", "ap_2010", "class_size", "demographics", "graduation", "hs_directory")

# create new dataframe and filter
duplicate_DBN <- ny_schools %>%
    map(mutate, is_dup = duplicated(DBN)) %>%
    map(filter, is_dup == "TRUE")

Only ap_2010 has a duplicate DBN value. If we use filter() take a look at rows containing the duplicated DBN value

In [53]:
ap_2010 %>%
    filter(DBN == "04M610")

DBN,SchoolName,AP Test Takers,Total Exams Taken,Number of Exams with scores 3 4 or 5,exams_per_student,high_score_percent
04M610,THE YOUNG WOMEN'S LEADERSHIP SCHOOL OF EAST HARLEM,41,55,29,1.341463,52.72727
04M610,YOUNG WOMEN'S LEADERSHIP SCH,NA,NA,NA,NA,NA


Only one of the rows, corresponding to the school name "THE YOUNG WOMEN'S LEADERSHIP SCHOOL OF EAST HARLEM", contains data. Therefore, we can remove the row with the school name "YOUNG WOMEN'S LEADERSHIP SCH"

In [54]:
# remove duplicate
ap_2010 <- ap_2010 %>%
    filter(SchoolName != "YOUNG WOMEN'S LEADERSHIP SCH")

# Outcome

- Simplifying data frames to contain only variables and observations needed for analysis
- Changing data types of multiple variables from character to numeric
- Creating new variables by calculating summary statistics from existing variables
- Checking for duplicated rows using the duplicated() function and functionals